# 特徴量エンジニアリング

モデルの性能は、アルゴリズム選択だけでなく「どの入力特徴量を作るか」で大きく変わります。
特徴量エンジニアリングは、元データをモデルが学びやすい表現に変換する作業です。

このノートでは、欠損値・カテゴリ変数・スケーリング・非線形変換・交互作用を、
実務で使う `Pipeline` / `ColumnTransformer` と結びつけて確認します。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

sns.set_theme(style="whitegrid", context="notebook")


## 1. 元データを観察する

まずは「どんな列があるか」を確認します。
特徴量設計の第一歩は、モデル前にデータの意味と型を把握することです。

以下のデータは教材用の疑似データです。
- `area`, `rooms`, `age`, `distance_to_station`, `floor` が価格に影響するという仮定で価格を生成
- `station` は立地プレミアムを表すカテゴリ列
- `noise` は観測できない要因（内装、周辺環境など）をまとめた誤差

また、実務に寄せるために数値列とカテゴリ列へ意図的に欠損を入れています。

In [ ]:
rng = np.random.default_rng(42)
n = 600

area = rng.normal(65, 18, n).clip(20, 150)
rooms = rng.integers(1, 6, n)
age = rng.integers(0, 40, n)
distance = rng.normal(18, 8, n).clip(1, 45)
station = rng.choice(["A", "B", "C", "D"], size=n, p=[0.30, 0.30, 0.25, 0.15])
floor = rng.integers(1, 16, n)

station_effect = pd.Series(station).map({"A": 180, "B": 120, "C": 60, "D": 20}).to_numpy()
noise = rng.normal(0, 35, n)
price = 45 + 4.2 * area + 16 * rooms - 2.3 * age - 3.8 * distance + station_effect + 2.8 * floor + noise

df = pd.DataFrame({
    "area": area,
    "rooms": rooms,
    "age": age,
    "distance_to_station": distance,
    "station": station,
    "floor": floor,
    "price": price,
})

# 欠損を意図的に作る
missing_idx_num = rng.choice(df.index, size=35, replace=False)
missing_idx_cat = rng.choice(df.index, size=20, replace=False)
df.loc[missing_idx_num, "distance_to_station"] = np.nan
df.loc[missing_idx_cat, "station"] = np.nan

df.head()


`info` と `isna` で型と欠損を確認します。
この確認を飛ばすと、後段で変換失敗やリークの原因になります。

In [ ]:
print(df.info())
print("\n欠損数:\n", df.isna().sum())


## 2. 変換前後で分布を比べる

歪んだ分布に対して対数変換が有効な場面があります。
ここでは駅距離の分布を `log1p` 変換前後で比較します。

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 3.6))

sns.histplot(df["distance_to_station"], bins=30, ax=axes[0], kde=True)
axes[0].set_title("original distance")

sns.histplot(np.log1p(df["distance_to_station"]), bins=30, ax=axes[1], kde=True)
axes[1].set_title("log1p(distance)")

plt.tight_layout()
plt.show()


## 3. ベースライン（最小限の前処理）

最初に単純なベースラインを作ります。
比較対象があると、後で追加した特徴量の効果を正しく評価できます。

In [ ]:
X = df.drop(columns=["price"])
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

num_cols = ["area", "rooms", "age", "distance_to_station", "floor"]
cat_cols = ["station"]

baseline_preprocess = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]), num_cols),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]), cat_cols),
])

baseline_model = Pipeline([
    ("preprocess", baseline_preprocess),
    ("regressor", LinearRegression()),
])

baseline_model.fit(X_train, y_train)
baseline_pred = baseline_model.predict(X_test)

print(f"baseline MAE: {mean_absolute_error(y_test, baseline_pred):.2f}")
print(f"baseline R2 : {r2_score(y_test, baseline_pred):.3f}")


## 4. 特徴量を追加する

次に、ドメイン知識を反映した特徴を追加します。

- `area_per_room`: 1部屋あたり面積。広さと間取りを1つの軸で表せる
- `is_new`: 築浅フラグ。築年数の閾値効果を捉えやすくする
- `log_distance`: 駅距離の対数。長い尾を圧縮して線形モデルで扱いやすくする

こうした手作業特徴は、モデルにとって有効な表現を直接与える手段です。

In [ ]:
def add_features(frame: pd.DataFrame) -> pd.DataFrame:
    out = frame.copy()
    out["area_per_room"] = out["area"] / np.maximum(out["rooms"], 1)
    out["is_new"] = (out["age"] <= 5).astype(int)
    out["log_distance"] = np.log1p(out["distance_to_station"])
    return out

feature_num_cols = [
    "area", "rooms", "age", "distance_to_station", "floor",
    "area_per_room", "is_new", "log_distance"
]
feature_cat_cols = ["station"]

feature_preprocess = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]), feature_num_cols),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]), feature_cat_cols),
])

feature_model = Pipeline([
    ("feature_builder", FunctionTransformer(add_features, validate=False)),
    ("preprocess", feature_preprocess),
    ("regressor", Ridge(alpha=1.0, random_state=42)),
])

feature_model.fit(X_train, y_train)
feature_pred = feature_model.predict(X_test)

print(f"feature MAE: {mean_absolute_error(y_test, feature_pred):.2f}")
print(f"feature R2 : {r2_score(y_test, feature_pred):.3f}")


## 5. 交互作用特徴を試す

線形モデルでは、列同士の掛け算項（交互作用）を入れると表現力が上がる場合があります。
`PolynomialFeatures(degree=2)` は、2次の項（例: `area^2`）や交互作用項（例: `area * rooms`）を自動生成します。

次元が増えやすいので、正則化や検証とセットで使います。

In [ ]:
poly_num_cols = ["area", "rooms", "age", "distance_to_station", "floor"]

poly_preview = PolynomialFeatures(degree=2, include_bias=False)
poly_preview.fit(X_train[poly_num_cols])
poly_names = poly_preview.get_feature_names_out(poly_num_cols)
print("num of polynomial features:", len(poly_names))
print("sample names:", poly_names[:12])

poly_preprocess = ColumnTransformer([
    ("num_poly", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("poly", PolynomialFeatures(degree=2, include_bias=False)),
        ("scaler", StandardScaler()),
    ]), poly_num_cols),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]), cat_cols),
])

poly_model = Pipeline([
    ("preprocess", poly_preprocess),
    ("regressor", Ridge(alpha=2.0, random_state=42)),
])

poly_model.fit(X_train, y_train)
poly_pred = poly_model.predict(X_test)

print(f"poly MAE: {mean_absolute_error(y_test, poly_pred):.2f}")
print(f"poly R2 : {r2_score(y_test, poly_pred):.3f}")


## 6. 木モデルとの比較

木ベースモデルはスケーリングに依存しにくく、
非線形関係も自動で取り込みやすいので、特徴量設計との相性を確認しやすいです。

In [ ]:
tree_preprocess = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
    ]), feature_num_cols),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]), feature_cat_cols),
])

tree_model = Pipeline([
    ("feature_builder", FunctionTransformer(add_features, validate=False)),
    ("preprocess", tree_preprocess),
    ("regressor", RandomForestRegressor(
        n_estimators=300,
        max_depth=10,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1,
    )),
])

tree_model.fit(X_train, y_train)
tree_pred = tree_model.predict(X_test)

print(f"random forest MAE: {mean_absolute_error(y_test, tree_pred):.2f}")
print(f"random forest R2 : {r2_score(y_test, tree_pred):.3f}")


## 7. リークを避けるための注意

特徴量エンジニアリングで最も危険なのはデータリークです。
例えば「テストデータを見てから欠損補完値を決める」「目的変数に依存する列を特徴に入れる」は禁止です。

`Pipeline` / `ColumnTransformer` を使って訓練データの手順を固定すれば、
推論時にも同じ変換を安全に適用できます。

次のセルでは、4つのモデルを 5-fold CV で同一基準比較します。
`cross_val_score` の `neg_mean_absolute_error` は「大きいほど良い」形式に合わせるため負符号付きで返るため、
表示時に `-scores.mean()` として通常の MAE（小さいほど良い）へ戻しています。

In [ ]:
models = {
    "baseline_linear": baseline_model,
    "feature_ridge": feature_model,
    "poly_ridge": poly_model,
    "tree_rf": tree_model,
}

rows = []
for name, model in models.items():
    # すべての前処理（特徴量追加を含む）をパイプライン内で実行
    scores = cross_val_score(
        model,
        X,
        y,
        cv=5,
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
    )
    rows.append({
        "model": name,
        "cv_mae_mean": -scores.mean(),
        "cv_mae_std": scores.std(),
    })

pd.DataFrame(rows).sort_values("cv_mae_mean")


## まとめ

特徴量エンジニアリングでは、

1. データの型と欠損を把握する
2. ベースラインを作る
3. 意味のある特徴を追加して比較する
4. CVで効果を確認する

という順序を崩さないことが重要です。

精度改善だけでなく、リークを避けて再現可能な前処理を作ることが、実務で最も価値のあるポイントです。